In [177]:
# Imports and Setup
import numpy as np
import heapq
import random
import itertools
import time
from collections import defaultdict, Counter



## Hearts AI: A Comparative Analysis of A*,Minimax, CSP and Rule-Based Strategies

This report presents an AI-based player for the Hearts card game, comparing
the performance of A* search with a simple rule-based approach.



## Game Rules & Challenges
Hearts is a trick-taking game where players aim to minimize penalty points.
- Each player is dealt 13 cards.
- Hearts and the Queen of Spades carry penalty points.
- Players must follow suit if possible; otherwise, they can play any card.
- The game ends when all cards are played, and the player with the lowest score wins.

### AI Challenges:
- **Hidden Information:** Players do not see opponents' hands.
- **Strategic Decision-Making:** Optimal card choices depend on future tricks.
- **Minimization of Penalty:** Avoiding high-penalty cards is crucial.



### A* Search
A* finds the optimal sequence of moves by evaluating:
- **G(n):** The total penalty accrued so far.
- **H(n):** A heuristic estimating future penalties.
- **F(n):** The sum of G(n) and H(n), guiding move selection.

### Rule-Based AI
A simpler approach that avoids penalty cards whenever possible.


### 🧩 CSP-Based Strategy

The CSP (Constraint Satisfaction Problem) approach aims to select cards that avoid constraint violations, namely:

1. **Avoid Hearts and the Queen of Spades**, since they carry penalties.
2. **Minimize the chance of winning tricks** by playing the lowest-value card.

If all cards are risky, the strategy relaxes constraints and picks the one with the least penalty. This mimics a form of defensive play in Hearts.

This method is simpler than A* or Minimax but still leverages logical filtering to avoid poor decisions.


## Minimax AI Implementation

The **Minimax algorithm** is a classical decision rule for minimizing the possible loss in a worst-case scenario. It is widely used in turn-based, zero-sum games like Hearts, where each player tries to minimize their own points (since in Hearts, lower scores are better).

### How Minimax Works in Hearts:
- The game is modeled as a tree of states.
- Each node alternates between the AI's move (maximizing) and the opponent's move (minimizing).
- The AI evaluates possible outcomes and chooses the path that minimizes the maximum points it might get stuck with.

Minimax is expected to outperform a rule-based approach in tightly constrained endgames or when decisions drastically affect point outcomes.

In [178]:
# Define Card Class
class Card:
    def __init__(self, suit, value):
        self.suit = suit
        self.value = value
    def __repr__(self):
        return f"{self.value} of {self.suit}"
    def __eq__(self, other):
        return self.suit == other.suit and self.value == other.value
    def __hash__(self):
        return hash((self.suit, self.value))

In [179]:
# Define Game State
class GameState:
    def __init__(self, hand, played_cards, parent=None):
        self.hand = hand
        self.played_cards = played_cards
        self.parent = parent

    def is_terminal(self):
        return len(self.hand) == 0

    def get_children(self):
        lead_suit = self.played_cards[-1].suit if self.played_cards else None
        legal_cards = [c for c in self.hand if c.suit == lead_suit] if lead_suit else self.hand
        if not legal_cards:
            legal_cards = self.hand

        children = []
        for card in legal_cards:
            new_hand = self.hand[:]
            new_hand.remove(card)
            new_played = self.played_cards + [card]
            children.append(GameState(new_hand, new_played))
        return children

    def cost(self):
        return sum(1 for c in self.played_cards if c.suit == 'hearts') + \
               sum(13 for c in self.played_cards if c.suit == 'spades' and c.value == 12)


In [180]:
# Limited Depth A* Search

# Improved A* Search with a unified heuristic
def a_star_search(initial_state, depth_limit=5, max_nodes=10000):
    import heapq, itertools
    open_list = []
    counter = itertools.count()

    # Keep track of (f-score, count, GameState, path_of_moves)
    heapq.heappush(open_list, (0, next(counter), initial_state, []))
    nodes_expanded = 0

    while open_list and nodes_expanded < max_nodes:
        _, _, current, path = heapq.heappop(open_list)
        nodes_expanded += 1

        if current.is_terminal() or len(current.played_cards) >= depth_limit:
            if path:
                return path[0]  # First move
            else:
                return random.choice(initial_state.hand)

        for child in current.get_children():
            move = child.played_cards[len(initial_state.played_cards)]  # Card played to reach this child
            new_path = path + [move]
            g = len(child.played_cards)
            h = improved_evaluate_state(child)
            f = g + (-h)
            heapq.heappush(open_list, (f, next(counter), child, new_path))

    print(f"⚠️ A* terminated early after expanding {nodes_expanded} nodes.")
    return random.choice(initial_state.hand)


# Unified evaluation as described
def improved_evaluate_state(state):
    hearts_penalty = sum(1 for c in state.played_cards if c.suit == 'H')
    q_spades_penalty = sum(13 for c in state.played_cards if c.suit == 'S' and (c.value == 'Q' or c.value == 12))
    remaining_cards = len(state.hand)
    return - (hearts_penalty + q_spades_penalty) - 0.1 * remaining_cards



In [181]:
# Rule-Based AI Implementation
def rule_based_ai_play(hand, played_cards):
    import random
    lead_suit = played_cards[-1].suit if played_cards else None
    legal_cards = [c for c in hand if c.suit == lead_suit] if lead_suit else hand
    if not legal_cards:
        legal_cards = hand
    return random.choice(legal_cards)


In [182]:
# Opponent move (simulated with rule-based logic)
def simulate_opponent_play(hand, played_cards):
    if not hand:
        return None
    card = rule_based_ai_play(hand, played_cards)
    hand.remove(card)
    played_cards.append(card)
    return card

In [183]:
# Minimax AI

# Improved Minimax with alpha-beta pruning and smarter evaluation
def minimax_decision(state, depth, alpha, beta, maximizing):
    if depth == 0 or state.is_terminal():
        return None, improved_evaluate_state(state)

    best_move = None
    if maximizing:
        max_eval = float('-inf')
        for child in state.get_children():
            _, eval = minimax_decision(child, depth - 1, alpha, beta, False)
            if eval > max_eval:
                max_eval = eval
                best_move = child.played_cards[len(state.played_cards)]
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return best_move, max_eval
    else:
        min_eval = float('inf')
        for child in state.get_children():
            _, eval = minimax_decision(child, depth - 1, alpha, beta, True)
            if eval < min_eval:
                min_eval = eval
                best_move = child.played_cards[len(state.played_cards)]
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return best_move, min_eval

# Aligned Evaluation Function for use in both A* and Minimax
def improved_evaluate_state(state):
    # Use played_cards (not the hand) to calculate the accumulated penalty
    hearts_penalty = sum(1 for c in state.played_cards if c.suit == 'H')
    # Adjust as needed: if your card representation for Queen is 'Q' or 12, check accordingly.
    q_spades_penalty = sum(13 for c in state.played_cards if c.suit == 'S' and (c.value == 'Q' or c.value == 12))
    # Optionally, add a term related to how many cards remain (if you want to prefer ending plays early)
    remaining_cards = len(state.hand)
    # Here, we simply return the negative penalty so that maximizing corresponds to minimizing cost.
    return - (hearts_penalty + q_spades_penalty) - 0.1 * remaining_cards





In [184]:
# CSP Strategy

# Improved CSP Strategy with basic constraints, aligned with evaluation logic
def csp_strategy(hand, played_cards):
    lead_suit = played_cards[-1].suit if played_cards else None
    legal_cards = [c for c in hand if c.suit == lead_suit] if lead_suit else hand
    if not legal_cards:
        legal_cards = hand

    # Avoid risky cards: if possible, avoid Q♠ and hearts.
    safe_cards = [c for c in legal_cards if not (c.suit == 'S' and (c.value == 'Q' or c.value == 12)) and c.suit != 'H']
    if safe_cards:
        return min(safe_cards, key=lambda c: c.value)
    return min(legal_cards, key=lambda c: c.value)


## Example Runs & Results
Simulating AI strategies with random hands.

In [185]:
def generate_deck():
    suits = ['hearts', 'diamonds', 'clubs', 'spades']
    ranks = [str(n) for n in range(2, 11)] + ['J', 'Q', 'K', 'A']
    return [Card(rank, suit) for suit in suits for rank in ranks]


## Evaluation Function

The `evaluate_ai` function takes a strategy and simulates a full hand using it. It tracks:
- Cards played by the AI
- Total cost (Hearts = 1 point, Queen of Spades = 13 points)
- Total time taken

This function helps us evaluate how good and efficient each strategy is.


In [186]:
def evaluate_final_state(played_cards):
    penalty = 0
    for card in played_cards:
        print("Sample played card:", played_cards[0])
        if card.suit == 'hearts':
            penalty += 1
        elif card.suit == 'spades' and (card.value == 'Q' or card.value == 12):
            penalty += 13
    return penalty



In [187]:
# Evaluate AI Strategy

def evaluate_ai(name, strategy_func, hand):
    hand_copy = hand[:]
    played = []
    start = time.time()

    while hand_copy:
        card = strategy_func(hand_copy, played)
        hand_copy.remove(card)
        played.append(card)

    end = time.time()
    cost = evaluate_final_state(played)
    return {'cost': cost, 'time': end - start}


## Running the Simulation

This function simulates a number of hands (e.g., 10 hands) for each AI strategy.

For each hand:
- A random hand is generated
- Each strategy plays it
- The cost and time are recorded

At the end, average performance is calculated and printed.


In [188]:
# Run Simulation

def run_simulation(num_trials=10):
    strategies = {
        'A*': a_star_wrapper,
        'Minimax': minimax_wrapper,
        'Rule-Based': rule_based_ai_play,
        'CSP': csp_wrapper
    }

    all_results = {name: [] for name in strategies}

    for trial in range(1, num_trials + 1):
        print(f"\n▶ Simulating hand {trial}/{num_trials}...")

        # 🃏 Generate ONE hand and copy for all AIs
        full_deck = generate_deck()
        random.shuffle(full_deck)
        hand = full_deck[:13]  # Assuming a 4-player game, take a 13-card hand

        for name, strategy in strategies.items():
            # 🧠 Give a fresh copy of the hand to each strategy
            print(f"  → {name} starting...")
            result = evaluate_ai(name, strategy, hand[:])
            all_results[name].append(result)
            print(f"  ✔ {name}: Cost = {result['cost']}, Time = {result['time']:.4f}s")
            all_results[name].append(result)
            print(f"{name}: Cost = {result['cost']}, Time = {result['time']:.4f}s")

    print("\n📈 Average Performance Over", num_trials, "Hands:")
    for name in strategies:
        avg_cost = sum(r['cost'] for r in all_results[name]) / num_trials
        total_time = sum(r['time'] for r in all_results[name])
        print(f"{name}: Avg Penalty = {avg_cost:.2f}, Total Time = {total_time:.4f}s")


In [189]:
from collections import defaultdict
import random

# --- Utility to ensure only Card objects are passed ---
def extract_card(result):
    if isinstance(result, tuple) and isinstance(result[0], Card):
        return result[0]
    elif isinstance(result, Card):
        return result
    else:
        raise ValueError(f"Invalid AI return type: {type(result)} - {result}")


# --- Smart AI play wrapper ---
def smart_ai_play(hand, played_cards, strategy="minimax"):
    state = GameState(hand, played_cards)
    if strategy == "a_star":
        return a_star_search(state, depth_limit=3, max_nodes=5000)
    elif strategy == "minimax":
        return minimax_decision(state, depth=3, alpha=float('-inf'), beta=float('inf'), maximizing=True)
    elif strategy == "csp":
        return csp_strategy(hand, played_cards)
    else:
        return rule_based_ai_play(hand, played_cards)

# --- Play one full game ---
def play_game(strategy="minimax", verbose=False):
    deck = generate_deck()
    random.shuffle(deck)
    hands = [deck[i::4] for i in range(4)]
    players = ["SmartAI", "RuleAI1", "RuleAI2", "RuleAI3"]
    player_scores = defaultdict(int)
    played_log = []

    for round_num in range(len(hands[0])):
        played_cards = []
        for i, hand in enumerate(hands):
            if not hand:
                continue
            if i == 0:  # Smart AI
                card = extract_card(smart_ai_play(hand, played_cards, strategy))
            else:
                card = extract_card(rule_based_ai_play(hand, played_cards))
            if card:
                played_cards.append(card)
                hand.remove(card)
                played_log.append((players[i], card.suit, card.value))
        if played_cards:
            winning_card = max(played_cards, key=lambda c: c.value)
            winner_index = played_cards.index(winning_card)
            winner_name = players[winner_index]
            player_scores[winner_name] += 1

    if verbose:
        for entry in played_log:
            print(f"{entry[0]} played {entry[2]} of {entry[1]}")
        print("Scores:", dict(player_scores))

    return max(player_scores, key=player_scores.get)

# --- Run multiple games ---
def simulate_games(num_games=100, strategy="minimax"):
    win_counter = defaultdict(int)
    for _ in range(num_games):
        winner = play_game(strategy=strategy)
        win_counter[winner] += 1
    return dict(win_counter)



In [190]:
import time

def compare_strategies(num_games=100):
    strategies = ["minimax", "a_star", "csp"]
    overall_results = {}

    print(f"\n{'='*10} Strategy Comparison ({num_games} Games Each) {'='*10}\n")

    for strategy in strategies:
        print(f"Simulating strategy: {strategy}...")
        start_time = time.time()
        results = simulate_games(num_games=num_games, strategy=strategy)
        duration = time.time() - start_time
        overall_results[strategy] = {
            "results": results,
            "time": duration
        }
        print(f"Completed in {duration:.2f} seconds\n")

    print(f"\n{'='*10} Results Summary {'='*10}")
    for strategy, data in overall_results.items():
        print(f"\nStrategy: {strategy.capitalize()} (Time: {data['time']:.2f}s)")
        sorted_results = sorted(data["results"].items(), key=lambda x: x[1], reverse=True)
        for player, wins in sorted_results:
            print(f"  {player:<10}: {wins}")

    return overall_results


In [191]:
results = compare_strategies(num_games=100)
for strat, res in results.items():
    print(f"\nStrategy: {strat}")
    for player, wins in res.items():
        print(f"  {player}: {wins}")


========== Strategy Comparison (100 Games Each) ==========

Simulating strategy: minimax...
Completed in 0.24 seconds

Simulating strategy: a_star...
Completed in 1.00 seconds

Simulating strategy: csp...
Completed in 0.02 seconds


========== Results Summary ==========

Strategy: Minimax (Time: 0.24s)
  SmartAI   : 51
  RuleAI1   : 35
  RuleAI2   : 7
  RuleAI3   : 7

Strategy: A_star (Time: 1.00s)
  SmartAI   : 43
  RuleAI1   : 33
  RuleAI2   : 17
  RuleAI3   : 7

Strategy: Csp (Time: 0.02s)
  SmartAI   : 40
  RuleAI1   : 32
  RuleAI2   : 21
  RuleAI3   : 7

Strategy: minimax
  results: {'RuleAI1': 35, 'SmartAI': 51, 'RuleAI2': 7, 'RuleAI3': 7}
  time: 0.23621535301208496

Strategy: a_star
  results: {'SmartAI': 43, 'RuleAI1': 33, 'RuleAI3': 7, 'RuleAI2': 17}
  time: 0.995905876159668

Strategy: csp
  results: {'RuleAI1': 32, 'SmartAI': 40, 'RuleAI2': 21, 'RuleAI3': 7}
  time: 0.017015457153320312
